In [ ]:
import codecs

import bs4

from bs4 import BeautifulSoup
import re
import numpy as np

In [ ]:
def create_chem_dict(vstr):
    chem_dict = dict()
    for n in range(len(vstr)):
        v1 = vstr[n].split(' => ')
        #pdb.set_trace()
        # get the chemical and the number unit
        match = re.match(r"([0-9]+) ([A-Z]+)", v1[1])
        mgr = match.groups()
        num = int(mgr[0])
        ch = mgr[1]
        chem_dict[ch] = [num, v1[0]]
        
        
    return chem_dict

In [ ]:
def count_ore(chem_dict):
    
    done = False
    
    vals = chem_dict['FUEL']
    #get list of chems needed
    chemc = vals[1].split(', ')
    chems = []
    chems = chem_components(chemc, chems, 1)
        
    orecnt = 0
    store = dict()
    cnt = 0
    #pdb.set_trace()
    while len(chems)>0:
        ch = chems[cnt][0]
        num = chems[cnt][1]
        #got to the chem dictionary to find out how much you have to get per unit.
        try:
            chc = chem_dict[ch]
            unt = chc[0]
        except:
            #this must be ore
            orecnt += num
            chems = chems[1:]
            continue
        #need to figure out how many to get
        #you need 'num', but you will get 'unt' numbers of this chemical.
        #check to see if you have any of a particular chemical in the "store"
        needmore = False
        try:
            nst = store[ch]
            if nst >= num:
            #there is more than enough in the store.  Just take those
                store[ch] -= num
            else:
                #note enough there, but can still just take all
                leftn = num - nst
                store[ch] = 0
                needmore = True
        except:
            #this chemical wasn't here at all, set the value to 0
            store[ch] = 0
            needmore = True
            leftn = num
            
        if needmore == True:
            
            if unt > leftn or unt == leftn:
                #only need 1 set
                vals = chem_dict[ch]
                #get list of chems needed
                chemc = vals[1].split(', ')
                chems = chem_components(chemc, chems, 1)
                store[ch] = store[ch]+(unt - leftn)
            else:
                #need more than 1 set
                nn = np.ceil(leftn/unt)
                vals = chem_dict[ch]
                #get list of chems needed
                chemc = vals[1].split(', ')
                chems = chem_components(chemc, chems, nn)
                store[ch] = store[ch]+((nn*unt)-leftn)
                
        #we addressed this one, and can erase it from the list
        chems = chems[1:]
                
    return orecnt, store         

In [ ]:
def chem_components(chemc, chems, nneed):
    
    for n in range(len(chemc)):
        #get the number needed of a chemical
        ch, num = get_chem_counts(chemc[n], chem_dict)
        chems.append([ch, num*nneed])
    return chems

In [ ]:
def get_chem_counts(chemstr, chem_dict):
    match = re.match(r"([0-9]+) ([A-Z]+)", chemstr)
    mgr = match.groups()
    num = int(mgr[0])
    ch = mgr[1]
    return ch, num

# Part 1

In [ ]:
f=codecs.open("input14.html", 'r')

In [ ]:
document= BeautifulSoup(f.read()).get_text()

vals = document.split('\n')
vals = vals[0:-1]

In [ ]:
chem_dict = create_chem_dict(vals)
count_ore(chem_dict)

Part 2 -- did not complete.  I was trying to figure out how to keep track of what is in the "store" after every iteration of using ore to make what I need for fuel.  I had a bunch of ore left after using what I could for fuel and I would have a bunch of left over stuff.  That stuff could be used up to make up parts for fuel.  I just didn't have the bookkeeping right, I think.  Need to spend more time.

In [ ]:
def count_ore(chem_dict):
    
    done = False
    
    vals = chem_dict['FUEL']
    #get list of chems needed
    chemc = vals[1].split(', ')
    chems = []
    chems = chem_components(chemc, chems, 1)
        
    orecnt = 0
    store = dict()
    cnt = 0
    #pdb.set_trace()
    while len(chems)>0:
        ch = chems[cnt][0]
        num = chems[cnt][1]
        #got to the chem dictionary to find out how much you have to get per unit.
        try:
            chc = chem_dict[ch]
            unt = chc[0]
        except:
            #this must be ore
            orecnt += num
            chems = chems[1:]
            continue
        #need to figure out how many to get
        #you need 'num', but you will get 'unt' numbers of this chemical.
        #check to see if you have any of a particular chemical in the "store"
        needmore = False
        try:
            nst = store[ch]
            if nst >= num:
            #there is more than enough in the store.  Just take those
                store[ch] -= num
            else:
                #note enough there, but can still just take all
                leftn = num - nst
                store[ch] = 0
                needmore = True
        except:
            #this chemical wasn't here at all, set the value to 0
            store[ch] = 0
            needmore = True
            leftn = num
            
        if needmore == True:
            
            if unt > leftn or unt == leftn:
                #only need 1 set
                vals = chem_dict[ch]
                #get list of chems needed
                chemc = vals[1].split(', ')
                chems = chem_components(chemc, chems, 1)
                store[ch] = store[ch]+(unt - leftn)
            else:
                #need more than 1 set
                nn = np.ceil(leftn/unt)
                vals = chem_dict[ch]
                #get list of chems needed
                chemc = vals[1].split(', ')
                chems = chem_components(chemc, chems, nn)
                store[ch] = store[ch]+((nn*unt)-leftn)
                
        #we addressed this one, and can erase it from the list
        chems = chems[1:]
                
    return orecnt, store         

In [ ]:
def count_all(chem_dict):
    
    done = False
    
    vals = chem_dict['FUEL']
    #get list of chems needed
    chemc = vals[1].split(', ')
    chems = []
    chems = chem_components(chemc, chems, 1)
        
    orecnt = 0
    store = dict()
    totalCount = dict()
    #totalUsed = dict()
    cnt = 0
    #pdb.set_trace()
    while len(chems)>0:
        ch = chems[cnt][0]
        num = chems[cnt][1]
        #got to the chem dictionary to find out how much you have to get per unit.
        try:
            chc = chem_dict[ch]
            unt = chc[0]
        except:
            #this must be ore
            orecnt += num
            chems = chems[1:]
            continue
        try:
            totalCount[ch] += num
        except:
            totalCount[ch] = num
        #need to figure out how many to get
        #you need 'num', but you will get 'unt' numbers of this chemical.
        #check to see if you have any of a particular chemical in the "store"
        needmore = False
        try:
            nst = store[ch]
            if nst >= num:
            #there is more than enough in the store.  Just take those
                store[ch] -= num
            else:
                #note enough there, but can still just take all
                leftn = num - nst
                store[ch] = 0
                needmore = True
        except:
            #this chemical wasn't here at all, set the value to 0
            store[ch] = 0
            needmore = True
            leftn = num
            
        if needmore == True:
            
            if unt > leftn or unt == leftn:
                #only need 1 set
                vals = chem_dict[ch]
                #get list of chems needed
                chemc = vals[1].split(', ')
                chems = chem_components(chemc, chems, 1)
                store[ch] = store[ch]+(unt - leftn)
            else:
                #need more than 1 set
                nn = np.ceil(leftn/unt)
                vals = chem_dict[ch]
                #get list of chems needed
                chemc = vals[1].split(', ')
                chems = chem_components(chemc, chems, nn)
                store[ch] = store[ch]+((nn*unt)-leftn)
                
        #we addressed this one, and can erase it from the list
        chems = chems[1:]
                
    return orecnt, store, totalCount         

In [ ]:
1000000000000/13312

In [ ]:
82892753 - 75120192.3076923

In [ ]:
def deplete_store(numore, minore, store, chem_dict):

    #how many times can you make 1 fuel out of the 1 trillion ore?
    numtimes = np.floor(numore/13312)
    oreleft = numore - (numtimes*minore)
    for key in store.keys():
        store[key] = store[key] * numtimes
        
    store['ore'] = oreleft
    
    #go through each chemical needed to make 1 fuel and and figure out how to get that from what is left.
    still_making = True
    while still_making = True:
        chems = chem_dict["FUEL"][1]
        #create a check list if you have a particular component
        gotit = np.zeros((len(chems)))
        n = 0
        while n < range(len(chems)) and still_making == True:
            #check for chemical availability in the store
            ch, num = get_chem_counts(chems[n], chem_dict)
            if store[ch] >= num:
                #there is enough here
                store[ch] -= num
                continue
            else:
                #there is not enough here.
                #take what's left and go from there.
                num = num - store[ch]
                store[ch] = 0
                #need num more of this chemical
                newchs = chem_dict[ch]
                if len(newchs[1]) == 1:
                    ch1, num1 = get_chem_counts(newchs[n][1], chem_dict)
                    if ch1 == 'ORE':
                        #this is the end of the line.  is there enough ore?
                        if store['ORE'] > num1:
                            #there is enough
                            store['ORE'] -= num1
                        else:
                            still_making = False
                            
                
    

In [ ]:
#how many times can you make 1 fuel out of the 1 trillion ore?

numtimes = np.floor(1000000000000/13312)

In [ ]:
store

In [ ]:
chem_dict